In [1]:
#Se debe tener instalado Ollama de manera local para que corra
from langchain_ollama import ChatOllama

llm= ChatOllama(model="llama3:8b")

In [2]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

Ahora definiremos una manera de guardar la información de forma vectorial

In [3]:
#De manera local
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Ahora crearemos la base de datos en base a Ley_Consumidor.csv

In [4]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def Cargar_documentos_csv(df):
    splitter = RecursiveCharacterTextSplitter( chunk_size = 1000, chunk_overlap=100, add_start_index=True)

    documents = []
    for index, columna in df.iterrows():
        Texto_articulo = columna['Texto_articulo']
        articulo = str(columna['Articulo'])
        
        if len(Texto_articulo) > 1000:
            splits = splitter.split_text(Texto_articulo)
            for split in splits:
                documents.append(
                    Document( page_content =split, metadata={'Articulo': articulo})
                )
        else:
            documents.append(
                Document( page_content =Texto_articulo, metadata= {'Articulo': articulo})
            )
    return documents

In [5]:
import pandas as pd

df = pd.read_csv("Ley_consumidor.csv")

i=0
while i!= 147:
    print(len(df.iloc[i]['Texto_articulo']))
    i = i + 1


5016
3128
1119
501
4903
4253
2801
292
491
645
977
2467
2078
672
1237
1943
606
264
1156
365
1418
1327
205
998
753
656
2787
508
352
2613
491
267
270
1984
1827
3580
577
11312
748
274
1285
2633
1048
910
326
480
1229
161
336
3163
4976
622
849
1398
5003
1950
1260
1502
1498
739
1216
318
755
321
2697
1350
1350
498
645
679
470
9007
203
363
915
1416
527
949
1194
298
473
200
1120
527
809
392
731
1789
2451
865
493
1650
549
1002
855
717
3875
573
10673
6253
4429
422
3400
3850
1761
1264
587
1373
265
1180
1353
649
2132
273
928
923
372
1090
1005
1629
1818
2510
707
653
2770
1023
883
2345
469
1529
2848
3826
2097
2433
3713
1302
472
912
1362
14324
1407
6464
841
1348
1380
102
14270


In [6]:
#Haremos split en chunks y cargaremos en documentos el DataFrame
docs = Cargar_documentos_csv(df)
print(f"Total characters: {len(docs[12].page_content)}")

Total characters: 501


Donde podemos ver que en la posicion 12 de docs encontramos el articulo N°2 ter del DataFrame

In [7]:
print(docs[12])

page_content='Las  Ley 21398 Art. 1 N° 2 D.O. 24.12.2021 normas contenidas en esta ley se interpretarán siempre en favor de los consumidores, de acuerdo con el principio pro consumidor, y, de manera complementaria, según las reglas contenidas en el párrafo 4° del Título Preliminar del Código Civil. TÍTU Ley N° 19.496 D.O. 07.03.1997 LO II      Disposiciones Ley N° 19.496 D.O. 07.03.1997  generales Párrafo 1º Ley N° 19.496 D.O. 07.03.1997      Los derechos Ley N° 19.496 D.O. 07.03.1997  y deberes del consumidor' metadata={'Articulo': 'Artículo 2 ter.-'}


Guardando los documentos

In [8]:
docs_ids = vector_store.add_documents(documents=docs)

print(docs_ids[:3])

['e4afbf6a-01e0-48e4-8250-b3bc76c5682c', '0ca45bc5-6619-47f7-ae59-3949f429f91c', '098e1b51-9c3d-43c9-b48d-f602cbd38b71']


Ahora iniciaremos con el Retrieval and Generation (RG)

Definimos el Prompt

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Responde con base en el siguiente **contexto legal** extraído de la Ley del Consumidor en Chile.

----------------
{context}
----------------

Pregunta:
{question}

Instrucciones:
- Utiliza lenguaje claro pero jurídicamente preciso.

Respuesta:
""")

Estado y funciones de RAG

In [25]:
from typing import TypedDict, List

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    pregunta = state["question"]
    retrieved_docs = vector_store.similarity_search(pregunta)
    return {"context": retrieved_docs}

def retrieve1(state: State):
    pregunta = state["question"]
    retrieved_docs1 = vector_store1.similarity_search(pregunta)
    return {"context": retrieved_docs1}
    

def generate(state: State):
    if not state["context"]:
        return {"answer": "No tengo suficiente contexto para responder."}
    
    # 🔁 Eliminar duplicados por contenido
    unique_docs = []
    seen_contents = set()
    for doc in state["context"]:
        if doc.page_content not in seen_contents:
            unique_docs.append(doc)
            seen_contents.add(doc.page_content)

    # Unir contenido limpio
    docs_content = "\n\n".join(
    f"[{doc.metadata.get('Articulo', 'Sin Artículo')}] {doc.page_content}"
    for doc in unique_docs
    )
    
    final_prompt = prompt.invoke({
        "question": state["question"],
        "context": docs_content
    })

    response = llm.invoke(final_prompt)
    
    # Agregar metadata de artículos como fuentes
    articles = set()
    for doc in unique_docs:
        metadata = getattr(doc, "metadata", {})
        articulo = metadata.get("Articulo")
        if articulo:
            articles.add(articulo)
    
    sources = "\n\nArtículos utilizados como fuente:\n" + "\n".join(f"- {a}" for a in sorted(articles))
    
    return {"answer": f"{response.content.strip()}\n\n{sources.strip()}"}

In [26]:
state = {"question":"¿Qué derechos tiene el consumidor frente a productos defectuosos?",
         "context": [],
         "answer": ""
        }

# 2. Llamar a la función `generate()` para obtener la respuesta
state.update(retrieve(state))
state.update(generate(state))

print(state["answer"])


Según la Ley del Consumidor en Chile (Ley N° 19.496), el consumidor tiene los siguientes derechos cuando se enfrenta con productos defectuosos:

1. **Reposición**: El consumidor puede optar por la reposición del producto, es decir, recibir un nuevo producto que cumpla con las características indicadas en el envase o empaque.
2. **Bonificación**: En caso de no ser posible la reposición, el consumidor puede elegir recibir bonificación en la compra de otro producto, lo que significa obtener una compensación por el valor del producto defectuoso.
3. **Devolución del precio pagado**: El consumidor tiene derecho a recibir devolución del precio que pagó en exceso, es decir, el valor del producto menos el valor del producto defectuoso.

Además, si el producto se vende con garantía, el plazo para ejercer este derecho será el establecido por la garantía, siempre y cuando sea mayor a los 6 meses siguientes a la fecha en que se recibió el producto (Artículo 21).

Es importante destacar que estos de

Ahora veremos si tenemos alguna mejora con el Csv limpio

In [12]:
import pandas as pd

df1 = pd.read_csv("Ley_consumidor_limpio.csv")

#Haremos split en chunks y cargaremos en documentos el DataFrame
docs1 = Cargar_documentos_csv(df1)
print(f"Total characters: {len(docs1[12].page_content)}")

Total characters: 995


In [13]:
print(docs1[12])

page_content='total del crédito, a su solo requerimiento. g) Acudir siempre ante el tribunal competente conforme a las disposiciones establecidas en esta ley. El proveedor debe NOTA 1 informar al consumidor de este derecho al celebrar el contrato y en el momento de surgir cualquier controversia, queja o reclamación. Toda estipulación en contrario constituye una infracción y se tendrá por no escrita. Sólo una vez surgido el conflicto, las partes podrán someterlo a mediación, conciliación o arbitraje. Los proveedores deben informar la naturaleza de cada uno de los mecanismos ofrecidos, los cuales serán gratuitos y sólo se iniciarán por voluntad expresa del consumidor, la que deberá constar por escrito. Un reglamento dictado por el Ministerio de Economía, Fomento y Turismo establecerá las normas que sean necesarias para la adecuada aplicación de los mecanismos a que se refiere este párrafo. Los proveedores financieros y no financieros podrán adscribir y ofrecer libremente el Sistema de So

In [14]:
vector_store1 = InMemoryVectorStore(embeddings)

docs_ids1 = vector_store1.add_documents(documents=docs1)

print(docs_ids1[:3])

['09e11e4d-b97e-4ef7-917f-62b01804eef2', '9d8e73cb-2f28-45b6-a56d-5d0986e909e8', '2609d733-a9ca-4c3a-b8d9-4bc2b305f3e8']


In [30]:
state = {"question":"¿Qué derechos tiene el consumidor frente a productos defectuosos?",
         "context": [],
         "answer": ""
        }

# 2. Llamar a la función `generate()` para obtener la respuesta
state.update(retrieve1(state))
state.update(generate(state))

print(state["answer"])


De acuerdo al contexto legal establecido en la Ley del Consumidor de Chile, el consumidor tiene los siguientes derechos frente a productos defectuosos:

1. **Reposición o bonificación**: El consumidor tiene derecho a optar entre la reposición del producto o la bonificación de su valor para adquirir otro producto (Artículo 19).
2. **Reparación gratuita**: En caso de defectos en los productos sujetos a normas de seguridad o calidad, el consumidor tiene derecho a optar entre la reparación gratuita o la reposición del producto o devolución del precio pagado (Artículo 20).
3. **Devolution of the price paid**: Si el producto no cumple con las especificaciones correspondientes, el consumidor puede optar por la devolución del precio pagado en exceso.
4. **Indemnización por daños ocasionados**: El consumidor tiene derecho a recibir indemnización por los daños causados por un producto defectuoso (Artículo 20).
5. **Plazo para ejercer derechos**: El consumidor debe ejercer sus derechos dentro de 